https://microsoft.github.io/autogen/dev/reference/python/autogen_ext.tools.graphrag.html
https://microsoft.github.io/graphrag/get_started/


In [35]:
# %pip install python-dotenv autogen_ext autogen_agentchat

from dotenv import load_dotenv
load_dotenv()
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_ext.tools.graphrag import GlobalSearchTool, LocalSearchTool
from autogen_agentchat.agents import AssistantAgent
import gradio as gr

# Initialize the OpenAI client
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

# Set up global search tool
global_tool = GlobalSearchTool.from_settings(settings_path="./settings.yaml")
local_tool = LocalSearchTool.from_settings(settings_path="./settings.yaml")

# Create assistant agent with the global search tool
assistant_agent = AssistantAgent(
    name="search_assistant",
    tools=[global_tool,local_tool],
    model_client=openai_client,
    system_message=(
        """You are a tool selector AI assistant using the GraphRAG framework. 
        Your primary task is to determine the appropriate search tool to call based on the user's query. 
        For broader, abstract questions requiring a comprehensive understanding of the dataset, call the 'global_search' function.
        For specific questions requiring a detailed, local search, call the 'local_search' function
        VMdata below is metadata for the VMs
    1. VM -> Name of the virtual machine.
    2. State -> Indicates whether the VM is ON or OFF.
    3. Status -> Describes the VM's condition (Normal or Special).
    4. Host -> The physical server hosting the VM.
    5. Cluster -> The group of hosts managing the VM.
    6. Provisioned_Space -> Total storage allocated to the VM.
    7. Used_Space -> Storage currently used by the VM.
    8. HostCPU -> CPU allocation on the host machine.
    9. HostMem -> Memory allocated to the VM.
    10. KN_SNC_DT -> Known since date of the VM record.
    11. Decom_date -> Planned decommissioning date of the VM."""
    ),
)

PermissionError: [Errno 1] Operation not permitted

In [31]:
import asyncio
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

def chat(question, history):
    response =  asyncio.run(
    assistant_agent.on_messages(
        [TextMessage(content=question, source="user")], CancellationToken()
    ))
    print(isinstance(response.chat_message.content, dict))
    if isinstance(response.chat_message.content, dict):
        return response.chat_message.content.get("answer")
    return response.chat_message.content

In [32]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


False


False


In [ ]:
answer =await chat('How many VMs are there in EAPSIPVXT9753 ?',[])

# Run a sample query
query = "What is EAPSIPVXT9753?"
await Console(assistant_agent.run_stream(task=query))


In [9]:
from datetime import datetime

# Collect records in a list
records = []

for index, row in vm_data_df.iterrows():
    record = row.to_dict()
    record_str = ', '.join(f"{key}: '{value}'" for key, value in record.items())
    source = "AppBrewery Virtual Machines"
    timestamp = datetime.now().strftime("%Y%m%d")
    records.append({
        "source": source,
        "text": record_str,
        "timestamp": timestamp
    })

# Create a dataframe from the records
records_df = pd.DataFrame(records)
print(records_df)
records_df.to_csv('records.csv', index=False)

                         source  \
0   AppBrewery Virtual Machines   
1   AppBrewery Virtual Machines   
2   AppBrewery Virtual Machines   
3   AppBrewery Virtual Machines   
4   AppBrewery Virtual Machines   
..                          ...   
95  AppBrewery Virtual Machines   
96  AppBrewery Virtual Machines   
97  AppBrewery Virtual Machines   
98  AppBrewery Virtual Machines   
99  AppBrewery Virtual Machines   

                                                 text timestamp  
0   VM: app96.kwbc.net, State: OFF, Status: Normal...  20250204  
1   VM: test55.kwbc.net, State: OFF, Status: Norma...  20250204  
2   VM: test48.kwbc.net, State: ON, Status: Specia...  20250204  
3   VM: db60.kwbc.net, State: OFF, Status: Normal,...  20250204  
4   VM: dev54.kwbc.net, State: OFF, Status: Normal...  20250204  
..                                                ...       ...  
95  VM: backup98.kwbc.net, State: OFF, Status: Nor...  20250204  
96  VM: infra44.kwbc.net, State: ON, Status: Norma.

In [6]:
import os 
os.getcwd()

'/Users/shivanksharma/Desktop/AI/Autogen0.4'